In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pprint import pprint 
import requests
from datetime import datetime, timedelta

from functools import reduce
import time

# Johns Hopkins Universtiry (JHU) github data
Two sets: 
1. Daily Reports - answer questions like "How many total cases in {location} on {date}?"
2. Time Series - answer questions like "How many cases per day in {location} thru {date_range}?"

https://github.com/CSSEGISandData/COVID-19

### A few notes:
1. All dates in the "current" are UTC
---------------------------

## Daily Reports 
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

**What**: Each file has 1 row that contains total cumulative totals from Jan 22 through this date\
**Keys**: Admin2 (city), Province_State, Country_Region\
**Update freq**: Daily\
**Use when**: 
1. You only care about the cumulative total numbers as of a specific date
2. You plot current/specific day's case numbers by either City/State/Country, State/Country, or Country
2. You want to map / chloropleth (inc. lat/long)

What: A count of the confirmed cases/deaths/recoveries from Nov 1, 2019 up until this date\
Granularity: City => State/Province => Country\

---------------------------

## Time Series  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

**What**: 1 row per Province/State, Country and 1 column per day since Jan 22 that contains that day's counts (not cumulative). Separate files for confirmed, deaths, recoveries.  \
**Keys**: State/Province, Country\
**Use when**: 
1. You want to plot daily case numbers by State/Province and/or Country
2. You want to map / chloropleth (inc. lat/long)

---------------------------

## Archived - Daily Reports
**What**: \
**Keys**: \
**Use when**: 

## Archived - Time Series
https://github.com/CSSEGISandData/COVID-19/tree/master/archived_data/archived_time_series

**What**: 1 row per City/State/Country and 1 column per day since Jan 22 that contains that day's counts\
**Keys**: State/Province, Country\
**Use when**: 
1. You want to plot daily case numbers by State and/or Country
2. You want to map / chloropleth (inc. lat/long)
* Example: On March 26, the March 25 csv had 66 columns - the 4 "keys" and 62 separate columns b/c there have have been 62 total days passed since Jan 22

--------------------------------

## Daily Reports - example of how to get the most recent CSV

--------------------------------

In [46]:
baseUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

# Step 1: Get the latest file
todaysDate = time.strftime("%m-%d-%Y")
todaysFile = todaysDate + ".csv"
dataUrl = baseUrl + todaysFile
finalFile = dataUrl;

print(f'Trying "{todaysFile}"...')
response = requests.head(dataUrl, timeout=5);
status_code = response.status_code;
reason = response.reason;

if(status_code != 200):
    print('.... error - 404 not found. Looking for yesterday file')

    days_to_subtract = 1;
    yestDate = datetime.today() - timedelta(days=days_to_subtract);
    yestDate = yestDate.strftime("%m-%d-%Y")
    yestFile = yestDate + ".csv"
    dataUrl = baseUrl + yestFile

    print(f'Trying "{yestFile}"...')
    response = requests.head(dataUrl, timeout=5);
    status_code = response.status_code
    reason = response.reason
    
    if(status_code != 200):
        finalFile = "";
        print(f'... Error! Unable to find yesterday file also')
    else:
        finalFile = dataUrl;
        print('... Successfully found yesterday file')

print('---------------------------------------')
print('')
print(finalFile);
print('')
print('---------------------------------------')

df = pd.read_csv(finalFile);

# Get rid of columns we don't want:
df = df.drop(df.columns[[0, 1, 2, 5, 6, 8, 9, 10, 11]], axis=1)

# Rename to make easier
df.columns = ["Country", "Date", "ConfirmedCases"];

# Convert from string "2020-03-26 23:48:35" to just the date only
df['Date'] = pd.to_datetime(df['Date'], errors='raise')
df['Date'] = df['Date'].dt.date

print(df.head())

Trying "03-27-2020.csv"...
.... error - 404 not found. Looking for yesterday file
Trying "03-26-2020.csv"...
... Successfully found yesterday file
---------------------------------------

https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-26-2020.csv

---------------------------------------
  Country        Date  ConfirmedCases
0      US  2020-03-26               3
1      US  2020-03-26               3
2      US  2020-03-26               2
3      US  2020-03-26              39
4      US  2020-03-26               1


In [62]:
# Ultimate goal: Get data in 3 column format: Date, Country, TotalConfirmedCasesThusFar

# Group by Country
gbAggs = df.groupby(['Country', 'Date']).agg({
        'ConfirmedCases': [
            np.sum
        ]
})

# Convert from groupby object to dataframe:
gbAggs = gbAggs.reset_index(level=['Country', 'Date'])

# Flatten the index by renaming the columns
gbAggs.columns = ["Country", "Date", "ConfirmedCases"];

gbAggs.sample(5)

,Country,Date,ConfirmedCases
180,Vietnam,2020-03-26,153
31,Chad,2020-03-26,3
77,Guinea-Bissau,2020-03-26,2
152,Singapore,2020-03-26,683
171,Turkey,2020-03-26,3629


--------------------------------

## Time Series

--------------------------------

In [2]:


# Step 2: Convert date columns to rows
df = df.drop(df.columns[[0, 2, 3]], axis=1)

# Step 3: Convert dates to invididual rows using melt()
key_columns = df.columns.to_list()[:1]
date_columns = df.columns.to_list()[1:]

df_clean = pd.melt(
    df
    , id_vars=key_columns
    , value_vars=date_columns
    , var_name='Date'
    , value_name='Confirmed'
)

# print(df_clean.head())

In [39]:
# Step 3: Group by Country - remove city/province
gbAggs = df_clean.groupby(['Country/Region', 'Date']).agg({
        'Confirmed': [
            np.sum
        ]
})

# Convert from groupby object to dataframe:
gbAggs = gbAggs.reset_index(level=['Country/Region', 'Date'])

# Flatten the index by renaming the columns
gbAggs.columns = ["Country", "Date", "ConfirmedCases"];
gbAggs['Date'] = pd.to_datetime(gbAggs['Date']) 
gbAggs['ConfirmedCases'] = pd.to_numeric(gbAggs['ConfirmedCases'], downcast='integer')
# gbAggs.head()

,Country,Date,ConfirmedCases
0,Afghanistan,2020-01-22,0
1,Afghanistan,2020-01-23,0
2,Afghanistan,2020-01-24,0
3,Afghanistan,2020-01-25,0
4,Afghanistan,2020-01-26,0


In [41]:
# Step 4: Remove all rows until a country has at least 100 rows
dfFinal = gbAggs[gbAggs["ConfirmedCases"] >= 100]
dfFinal = dfFinal.sort_values(["Country", "Date"]);
# dfFinal = dfFinal.set_index("Country", "Date");
# dfFinal.head()
# dfFinal[dfFinal["Country"] == 'US'].tail()

,Country,Date,ConfirmedCases
176,Algeria,2020-03-21,139
177,Algeria,2020-03-22,201
178,Algeria,2020-03-23,201
239,Andorra,2020-03-22,113
240,Andorra,2020-03-23,113


# Target dataframe column format
1. 2000-01-01, 'Coca-Cola', '72537'
2. 2000-01-01, 'Microsoft', '70196'

# Target json data format: 
1. {date: 2000-01-01, name: "Coca-Cola", category: "Beverages", value: 72537}
2. {date: 2000-01-01, name: "Microsoft", category: "Technology", value: 70196}

In [44]:
# orient 'records' creates one element per row w no index value 
print(dfFinal.head().to_json(orient='records', date_format='iso'))

# Export to file
jsonFile = 'temp.json';
dfFinal.to_json(jsonFile, orient='records');
# https://jsonformatter.curiousconcept.com/

[{"Country":"Algeria","Date":"2020-03-21T00:00:00.000Z","ConfirmedCases":139},{"Country":"Algeria","Date":"2020-03-22T00:00:00.000Z","ConfirmedCases":201},{"Country":"Algeria","Date":"2020-03-23T00:00:00.000Z","ConfirmedCases":201},{"Country":"Andorra","Date":"2020-03-22T00:00:00.000Z","ConfirmedCases":113},{"Country":"Andorra","Date":"2020-03-23T00:00:00.000Z","ConfirmedCases":113}]


# Done - now go use the json file in d3